
<center> <h1> <font color='blue'> Notebook 2 : "Clean_data_and_Save_to_csv"</font></h1> <h2> Group 4 </h2> </center> 

## Plan of this notebook : 

1. [About this notebook](#1)  <br>
<br>

2. [Prepare data](#2) <br>
<br>
3. [Group data per zip code and clean non existant zip codes](#3) <br>
<br>
4. [Define useful processing functions](#4) <br>
<br>
5. [Save grouped dataframes](#5) <br>
<br>
6. [Save clean dataframes](#6) <br>
<br>
7. [Save lemmatized dataframes](#7) <br>




<a class="anchor" id="1"></a> 

## 1. About this notebook

In this notebook, we will try to prepare our data for further work. <br>
<br>
First, we will clean the data by removing punctuation and numbers. Then we will remove stopwords that are the most used in the French language.
We will save the work in a CSV file **"clean_environement.csv".**

<br>

After that, We will add lemmatization and use Spacy for this since it is the most efficient tool now. We will keep only words that are NOUNS and proper nouns since environmental problems would be expressed as nouns by users. Spacy also allows this very easily.
The new data will be saved in **"lemmatized_environement.csv"**

<br>
Later, we will group the data per zip and cleans the non existant zip-codes, we will clean the rows with zip codes that do not exist in France, then we will group the responses for each zip code. So that we will group the problems and answers of each region in the same row.
We will save the data in **"grouped_lemmetized.csv"**


## Import libraries

In [1]:
import pandas as pd
import numpy as np
import nltk
from pprint import pprint

nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import  word_tokenize

# spacy for lemmatization
import spacy
!python -m spacy download fr


import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


import re
import os
pd.set_option('max_colwidth', 400)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/siwarabbes/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/siwarabbes/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')
✔ Linking successful
/Users/siwarabbes/anaconda3/lib/python3.7/site-packages/fr_core_news_sm -->
/Users/siwarabbes/anaconda3/lib/python3.7/site-packages/spacy/data/fr
You can now load the model via spacy.load('fr')


<a class="anchor" id="2"></a> 

## 2. Prepare data

In [6]:
envr_df = pd.read_csv('LA_TRANSITION_ECOLOGIQUE.csv', low_memory=False)
envr_df.head()

,id,reference,title,createdAt,publishedAt,updatedAt,trashed,trashedStatus,authorId,authorType,...,"QUXVlc3Rpb246MTUw - Qu'est-ce qui pourrait vous inciter à changer vos comportements comme par exemple mieux entretenir et régler votre chauffage, modifier votre manière de conduire ou renoncer à prendre votre véhicule pour de très petites distances ?",QUXVlc3Rpb246MTUx - Quelles seraient pour vous les solutions les plus simples et les plus supportables sur un plan financier pour vous inciter à changer vos comportements ?,"QUXVlc3Rpb246MTUy - Par rapport à votre mode de chauffage actuel, pensez-vous qu'il existe des solutions alternatives plus écologiques ?","QUXVlc3Rpb246MTUz - Si oui, que faudrait-il faire pour vous convaincre ou vous aider à changer de mode de chauffage ?","QUXVlc3Rpb246MTU0 - Avez-vous pour vos déplacements quotidiens la possibilité de recourir à des solutions de mobilité alternatives à la voiture individuelle comme les transports en commun, le covoiturage, l'auto-partage, le transport à la demande, le vélo, etc. ?","QUXVlc3Rpb246MTU1 - Si oui, que faudrait-il faire pour vous convaincre ou vous aider à utiliser ces solutions alternatives ?","QUXVlc3Rpb246MjA3 - Si non, quelles sont les solutions de mobilité alternatives que vous souhaiteriez pouvoir utiliser ?",QUXVlc3Rpb246MTU3 - Et qui doit selon vous se charger de vous proposer ce type de solutions alternatives ?,QUXVlc3Rpb246MTU4 - Que pourrait faire la France pour faire partager ses choix en matière d'environnement au niveau européen et international ?,QUXVlc3Rpb246MTU5 - Y a-t-il d'autres points sur la transition écologique sur lesquels vous souhaiteriez vous exprimer ?
0,UHJvcG9zYWw6ZmZlN2MwNzktMWUyMC0xMWU5LTk0ZDItZmExNjNlZWIxMWUx,2-4,transition écologique,2019-01-22 09:37:49,2019-01-22 09:37:49,NaN,False,NaN,VXNlcjoxMTQwMTc0YS0xZTFmLTExZTktOTRkMi1mYTE2M2VlYjExZTE=,Citoyen / Citoyenne,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Enseignement du tri sélectif à l'école,Multiplier les centrales géothermiques
1,UHJvcG9zYWw6M2RmZmUxZjMtMWUyMS0xMWU5LTk0ZDItZmExNjNlZWIxMWUx,2-5,La surpopulation,2019-01-22 09:39:33,2019-01-22 09:39:33,NaN,False,NaN,VXNlcjpjOWYxZWQ1NS0xYzEwLTExZTktOTRkMi1mYTE2M2VlYjExZTE=,Citoyen / Citoyenne,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mettre en oeuvre au niveau national ses engagements internationaux,"Les problèmes auxquels se trouve confronté l’ensemble de la planète et que dénoncent, dans le plus parfait désordre, les gilets jaunes de France ne sont-ils pas dus, avant tout, à la surpopulation mondiale ? Cette population est passée d’1,5 milliards d’habitants en 1900 à 7 milliards en 2020 et montera bientôt à 10 milliards vers 2040. Avec les progrès de la communication dans ce village mon..."
2,UHJvcG9zYWw6NDg1YWUxYjItMWUyMS0xMWU5LTk0ZDItZmExNjNlZWIxMWUx,2-6,climat,2019-01-22 09:39:50,2019-01-22 09:39:50,NaN,False,NaN,VXNlcjozZjlhNzAwOS0xYTc2LTExZTktOTRkMi1mYTE2M2VlYjExZTE=,Citoyen / Citoyenne,...,"développer les transports en commun ,",NaN,Non,NaN,Oui,NaN,Les transports en commun|L'auto partage|Le transport à la demande,NaN,NaN,NaN
3,UHJvcG9zYWw6NTk2Nzk5MmItMWUyMS0xMWU5LTk0ZDItZmExNjNlZWIxMWUx,2-7,POLLUTION AIR EAU,2019-01-22 09:40:19,2019-01-22 09:40:19,NaN,False,NaN,VXNlcjozOWQwNzJjNC0xZDEwLTExZTktOTRkMi1mYTE2M2VlYjExZTE=,Citoyen / Citoyenne,...,NaN,Plus de transports publics dans les petites communes pour rejoindre les grandes agglomérations proches.,Non,NaN,Non,Le co-voiturage ne correspond pas toujours aux demandes. Les lieux de desserte sont loin des centre-villes.,Les transports en commun,L'Etat,"Elle n'a aucun pouvoir. Impossible de contraindre l'afrique, l'asie, etc à adopter des mesures anti-pollution.",NaN
4,UHJvcG9zYWw6ODU5NWMyOTYtMWUyMS0xMWU5LTk0ZDItZmExNjNlZWIxMWUx,2-8,Economie vs Ecologie,2019-01-22 09:41:33,2019-01-22 09:41:33,NaN,False,NaN,VXNlcjo3M2YxN2NlZS0xZDRiLTExZTktOTRkMi1mYTE2M2VlYjExZTE=,Citoyen / Citoyenne,...,"Aménagement de piste cyclable, développement du covoiturage, des ""vrais""bus propres, une aide pour un mode

In [7]:
# Change column names
envr_df.columns= ['id', 'reference', 'title', 'createdAt', 'publishedAt', 'updatedAt',
       'trashed', 'trashedStatus', 'authorId', 'authorType', 'authorZipCode', 'problemes', 'reponse_probleme', 'q3','q4','q5','q6','q7','q8','q9','q10','q11','q12','q13','q14','q15','q16']

# Remove the nan values
envr_df.fillna('',inplace=True)

In [8]:
# We convert our data to type string
envr_df[['problems', 'reponse_probleme', 'q3','q4','q5','q6','q7','q8','q9','q10','q11','q12','q13','q14','q15','q16']] = envr_df[['problemes', 'reponse_probleme', 'q3','q4','q5','q6','q7','q8','q9','q10','q11','q12','q13','q14','q15','q16']].astype(str)

In [9]:
# We will focus only on zipcode,title, the response of Q3 which concerns the ecological problems and the response on Q4 which concerns the solution proposed to these problems
reduced_dataset_df = envr_df[['authorZipCode', 'title', 'problems', 'reponse_probleme']]

<a class="anchor" id="3"></a> 

## 3. Group data per zip code and clean non existant zip codes

We got the data containing all zip codes in France from INSEE [website]('https://www.data.gouv.fr/en/datasets/correspondance-code-insee-code-postal/)

In [12]:
cities = pd.read_csv('zip_codes.csv', low_memory=False, delimiter=';')
valid_zip_codes = list(cities['Code_postal'])
# Delete rows containing non valid zip codes
reduced_dataset_df = reduced_dataset_df.loc[reduced_dataset_df['authorZipCode'].isin(valid_zip_codes)]
# Group data with the same zip code and concatenate responses
reduced_dataset_df = reduced_dataset_df.groupby('authorZipCode',as_index=False).agg(lambda x :' '.join(x))
reduced_dataset_df.head()

,authorZipCode,title,problems,reponse_probleme
0,1000,"Une répartition plus juste des efforts concernant la transition écologique Valoriser financièrement les actions écologiques Développement du ferroutage, plantation d'arbres subventionner les agriculteurs pour une transition ecologique suppression des pesticides ,, Sortir les investissements écolo du budget Passer d'une transition écologique imposée à une transition écologique voulue. ""Transiti...","Les dérèglements climatiques (crue, sécheresse) La biodiversité et la disparition de certaines espèces La pollution de l'air Les dérèglements climatiques (crue, sécheresse) Les dérèglements climatiques (crue, sécheresse) Les dérèglements climatiques (crue, sécheresse) La biodiversité et la disparition de certaines espèces il n'y a pas qu'un seul problème; tous sont importants mais la pollution...","L'effort doit être soutenu tant par les particuliers que par les pollueurs les plus importants (industries) et les administrations. La taxe carbone devrait toucher tous ces acteurs en fonction des émissions polluantes de manière exponentielle, et son intégralité être affectée à l'accélération de la transition écologique : installation de parcs éoliens en zone venteuse et en mer ; centrales sol..."
1,1090,Rendre plus importante la vie économique des petits villages Avec quels moyens changer sa chaudière au fioul et sa voiture diesel?? Décentralisation et relocalisation des productions véhicule électrique écologie cause mondiale Pour une transition écologique durableet solidaire Urgence climatique avis de retraités,"Les dérèglements climatiques (crue, sécheresse) La pollution des océans La pollution de l'air Les dérèglements climatiques (crue, sécheresse) Les dérèglements climatiques (crue, sécheresse) toutes les réponses ci-dessus La production d'energie fossile pour le carburant et le plastique Les dérèglements climatiques (crue, sécheresse)","Travailler sur la source du problème: réduire l'utilisation énergétique. Interdire tout les plastiques non biodégradables, taxer assez lourdement les paquebots de croisière La centralisation dans les grandes villes encourage la désertification des services, l'augmentation de la nécessité de se déplacer en voiture et l'augmentation des bouchons dans les grandes villes. PRENDRE DES SANCTIONS CO..."
2,1100,"Pollution Stop aux energies fosiles Il faut se sevrer du pétrole Développement durable NON A L'ECOLOGIE PUNITIVE ET A L'ENFUMAGE DES ESPRITS Transport des marchandises conscience écologique INDIVIDUELLE , POLITIQUE et EUROPEENNE Vers une nouvelle façon de vivre Magnum Tout le monde est concerné, à tous les niveaux. Mais la contribution doit être proprtionnelle à la pollution engendrée. Préserv...","La pollution de l'air Les dérèglements climatiques (crue, sécheresse) notre consommation de petrole. L'agriculture chimique La biodiversité et la disparition de certaines espèces La biodiversité et la disparition de certaines espèces Les dérèglements climatiques (crue, sécheresse) La biodiversité et la disparition de certaines espèces Les dérèglements climatiques (crue, sécheresse) Les dérègle...","Taxer tous les poids lourds sans exception , réaménagés tous les fleuves capable de supporter le transport fluvial . Interdire le centre ville a tous les véhicules a moteur. Remplacer tous les véhicules de livraisons par des charrettes tirer par des chevaux ! privilégier l’hydrogène Les experts s'accordent à dire que nous n'aurons plus de pétrole dans 20 ans et nous ne sommes pas prêts à nous ..."
3,1110,Pour une bonne santé Proposition pour l'international sans doute un peu utopiste Justice écologique et ruralité Changer nos comportements revoir nos priorités Civisme et bon sens Pollution de l'air Réchauffement climatique - pollution de l'air PENALISER CEUX QUI SOUILLENT LA NATURE santé publique et environnement,"Les dérèglements climatiques (crue, sécheresse) Tous ces problèmes réunis, il serait difficile d'en faire émerger un qui soit plus urgent ou plu

<a class="anchor" id="4"></a> 

## 4. Define useful processing functions


In [27]:
def clean(X):
    # X, column data
    pd_list = list(X)
    pd_list = [it.lower() for it in pd_list] # to lowercase
    # We noticed that people use o instead of ô frequently so we replaced it to get more robust text
    pd_list = [re.sub(r'ô','o',it) for it in pd_list] 
    # Same thing with é ê è contributers often misspell these characters so we replaced it with simple e
    pd_list = [re.sub(r'[éêè]','e',it) for it in pd_list] 
    # Delete non alphabetic characters
    pd_list = [re.sub(r'([^a-zéèûôàêô])',' ',it) for it in pd_list]
    return pd_list

In [14]:
def tokenization(X_list):
    tokenized_list = [word_tokenize(it) for it in X_list]
    return tokenized_list

In [15]:

#We added this list to give more sens to our results.
other_stopwords = ['les','a','afin','alors',  'plus', 'moins', 'faut', 'tout', 'tous', 'prends', 'tre', 'si','non','doit','avoir','comme','trop','leurs','faire','ils','peut','bien','aussi','cela','gens', 'sans', 'car', 'très', 'fait', '\'', 'qu\'', 'd\'', 'l\'', '"', 'être', 'mai', 'faudrait']

stopWords = stopwords.words('french')
stopWords.extend(other_stopwords)
stopWords = set(stopWords)

def remove_stopwords(X_list):
    filtered_list =[]
    for it in X_list:
        
        temp = []
        for word in it:
            if(word not in stopWords):
                temp.append(word)
        filtered_list.append(temp)
        temp=[]
     
    return filtered_list

In [16]:
### Function that cleans and removes stopwords
def clean_remove_stopwords(column_data):
  
    non_stopwords_data = remove_stopwords(tokenization(clean(column_data)))
    non_tokenized = []
    for l in non_stopwords_data:
    non_tokenized.append(' '.join(l))
  
  return non_tokenized



In [17]:
# Function that lemmatized data. We used Spacy 
def lemmetize(column_data):
    nlp = spacy.load('fr', disable=['parser', 'ner'])
    rep_lemmas = []
    for i, reponse in enumerate(column_data):
        if(i % 25000 == 0) :
            print('{} processed colmuns now '.format(i))
        if type(reponse) == str:
            doc = nlp(reponse)
            #We only keep lemmas which are nouns maybe we have to add VERB.
            # lemmas = [token.lemma_ if token.pos_ in ['NOUN', 'PROPN', 'ADJ'] else ''  for token in doc]
            # Keep only nouns and proper nouns in our text
            lemmas = [token.lemma_ for token in doc if token.pos_ in ['NOUN', 'PROPN']]
        else:
            lemmas = ''
        rep_lemmas.append(' '.join(lemmas))
    return rep_lemmas


In [18]:
## test lemmatization
text = ['je suis transitions écologiques', 'une éxpérience écologie naturelle trasformation']
print(lemmetize(text))

0 processed colmuns now 
['transition', 'éxpérience écologie trasformation']


<a class="anchor" id="5"></a> 

## 5. Save grouped dataframes

In [19]:
grouped_df = pd.DataFrame()

grouped_df['authorZipCode'] = reduced_dataset_df['authorZipCode']
clean_column = clean_remove_stopwords(reduced_dataset_df['title'])
grouped_df['title'] = clean_column
clean_column = clean_remove_stopwords(reduced_dataset_df['problems'])
grouped_df['problems'] = clean_column
clean_column = clean_remove_stopwords(reduced_dataset_df['reponse_probleme'])
grouped_df['reponse_probleme'] = clean_column

grouped_df.head()

,authorZipCode,title,problems,reponse_probleme
0,1000,répartition juste efforts concernant transition écologique valoriser financièrement actions écologiques développement ferroutage plantation arbres subventionner agriculteurs transition ecologique suppression pesticides sortir investissements écolo budget passer transition écologique imposée transition écologique voulue transition déjà tard arrêter essayer argent sous prétexte protection enviro...,dérèglements climatiques crue sécheresse biodiversité disparition certaines espèces pollution air dérèglements climatiques crue sécheresse dérèglements climatiques crue sécheresse dérèglements climatiques crue sécheresse biodiversité disparition certaines espèces seul problème importants pollution air dérèglement climatique découle importants dérèglements climatiques crue sécheresse pollution ...,effort soutenu tant particuliers pollueurs importants industries administrations taxe carbone devrait toucher acteurs fonction émissions polluantes manière exponentielle intégralité affectée accélération transition écologique installation parcs éoliens zone venteuse mer centrales solaires zones rurales sud utilisation énergie géothermique réduire considérablement empreinte carbone taxer produi...
1,1090,rendre importante vie économique petits villages quels moyens changer chaudière fioul voiture diesel décentralisation relocalisation productions véhicule électrique écologie cause mondiale transition écologique durableet solidaire urgence climatique avis retraités,dérèglements climatiques crue sécheresse pollution océans pollution air dérèglements climatiques crue sécheresse dérèglements climatiques crue sécheresse toutes réponses ci dessus production energie fossile carburant plastique dérèglements climatiques crue sécheresse,travailler source problème réduire utilisation énergétique interdire plastiques biodégradables taxer assez lourdement paquebots croisière centralisation grandes villes encourage désertification services augmentation nécessité déplacer voiture augmentation bouchons grandes villes prendre sanctions commerciales envers pays pollueurs diminuer consommation stopper extraction énergies fossiles donn...
2,1100,pollution stop energies fosiles sevrer pétrole développement durable ecologie punitive enfumage esprits transport marchandises conscience écologique individuelle politique europeenne vers nouvelle fa vivre magnum monde concerné niveaux contribution proprtionnelle pollution engendrée préservons maison urgence protection planète etre toute ere priorité prendre recul analyser fa pragamatique glob...,pollution air dérèglements climatiques crue sécheresse consommation petrole agriculture chimique biodiversité disparition certaines espèces biodiversité disparition certaines espèces dérèglements climatiques crue sécheresse biodiversité disparition certaines espèces dérèglements climatiques crue sécheresse dérèglements climatiques crue sécheresse biodiversité disparition certaines espèces quat...,taxer poids lourds exception réaménagés fleuves capable supporter transport fluvial interdire centre ville véhicules moteur remplacer véhicules livraisons charrettes tirer chevaux privilégier hydrogène experts accordent dire pétrole ans prêts passer complètement pétrole contraire tellement dépendants pétrole lorsqu économie mondiale effondrera monde sombrera chaos inimaginable choix dès présen...
3,1110,bonne santé proposition international doute peu utopiste justice écologique ruralité changer comportements revoir priorités civisme bon sens pollution air réchauffement climatique pollution air penaliser ceux souillent nature santé publique environnement,dérèglements climatiques crue sécheresse problèmes réunis difficile émerger urgent important autres pollution air dérèglements climatiques crue sécheresse dérèglements climatiques crue sécheresse biodiversité disparition certaines espèces pollution air dérèglements climatiques crue sécheresse saletes jetes certains citoyens anture cann

In [20]:
grouped_df.to_csv('grouped_filtered_not_cleaned_environement.csv')

<a class="anchor" id="6"></a> 

## 6. Save clean dataframes

In [21]:
clean_df = pd.DataFrame()
clean_df['authorZipCode'] = envr_df['authorZipCode']
clean_column = clean_remove_stopwords(envr_df['title'])
clean_df['title'] = clean_column
clean_column = clean_remove_stopwords(envr_df['problems'])
clean_df['problems'] = clean_column
clean_column = clean_remove_stopwords(envr_df['reponse_probleme'])
clean_df['reponse_probleme'] = clean_column

clean_df.head()

,authorZipCode,title,problems,reponse_probleme
0,97231,transition écologique,,
1,57000,surpopulation,problèmes auxquels trouve confronté ensemble planète dénoncent parfait désordre gilets jaunes france dus avant surpopulation mondiale cette population passée milliards habitants milliards montera bientot milliards vers progrès communication village mondial chaque individu fin fond asie fin fond afrique passant quartiers campagnes pays aspire vivre peu bl mer mal lotis concitoyens logement nour...,problèmes auxquels trouve confronté ensemble planète dénoncent parfait désordre gilets jaunes france dus avant surpopulation mondiale cette population passée milliards habitants milliards montera bientot milliards vers progrès communication village mondial planète chaque individu fin fond asie fin fond afrique passant quartiers campagnes pays aspire vivre peu bl mer mal lotis concitoyens logem...
2,34140,climat,dérèglements climatiques crue sécheresse,éviter inondations obliger riverains communes nettoyer secteur
3,17400,pollution air eau,pollution air,taxer gros pollueurs entreprises bateaux marchands routiers etc
4,35430,economie vs ecologie,biodiversité disparition certaines espèces,changer mode vie impulser nouvelle économie agricole


In [22]:
clean_df.to_csv('clean_environement.csv')

<a class="anchor" id="7"></a> 

## 7. Save lemmatized dataframes

In [23]:
lemmetized_df = pd.DataFrame()
lemmetized_df['authorZipCode'] = clean_df['authorZipCode']
lemmetized_df['title'] = lemmetize(clean_df['title'])
lemmetized_df['problems'] = lemmetize(clean_df['problems'])
lemmetized_df['reponse_probleme'] = lemmetize(clean_df['reponse_probleme'])


lemmetized_df.head()

0 processed colmuns now 
25000 processed colmuns now 
50000 processed colmuns now 
75000 processed colmuns now 
0 processed colmuns now 
25000 processed colmuns now 
50000 processed colmuns now 
75000 processed colmuns now 
0 processed colmuns now 
25000 processed colmuns now 
50000 processed colmuns now 
75000 processed colmuns now 


,authorZipCode,title,problems,reponse_probleme
0,97231,transition,,
1,57000,surpopulation,problème ensemble gilet france devoir surpopulation population milliard milliard bientot milliard progrès communication village individu fin fond fin fond campagne pays nourriture bien consommation mère problème cas solution problème croissance partage richesse partage partage eau protection déforestation règlement stabilisation concurrence aise france débat,problème ensemble gilet france devoir surpopulation population milliard milliard bientot milliard progrès communication village individu fin fond fin fond campagne pays nourriture bien consommation mère problème cas solution problème croissance partage richesse partage partage eau protection déforestation règlement stabilisation concurrence aise france débat
2,34140,climat,dérèglement crue,inondation riverain secteur
3,17400,pollution air eau,pollution air,pollueur entreprise
4,35430,economie ecologie,biodiversité disparition espèce,mode vie économie


In [31]:
lemmetized_df.to_csv('lemmatized_environement.csv')

In [25]:
### Group and lemmetize
grouped_lemm_df = pd.DataFrame()

grouped_lemm_df['authorZipCode'] = grouped_df['authorZipCode']
grouped_lemm_df['title'] = lemmetize(grouped_df['title'])
grouped_lemm_df['problems'] = lemmetize(grouped_df['problems'])
grouped_lemm_df['reponse_probleme'] = lemmetize(grouped_df['reponse_probleme'])


grouped_lemm_df.head()

0 processed colmuns now 
0 processed colmuns now 
0 processed colmuns now 


,authorZipCode,title,problems,reponse_probleme
0,1000,répartition transition action ferroutage plantation agriculteur transition suppression pesticide investissement budget transition transition transition argent prétexte protection environnement procédure déchet agriculteur industrie aise mutation véhicule culpabilisation ecologie police sauvon problème calendrier éducation transition circuit système consigne transition economie écologie punitiv...,dérèglement crue biodiversité disparition espèce pollution air dérèglement crue dérèglement crue dérèglement crue biodiversité disparition espèce problème pollution découle dérèglement crue pollution air emballage dérèglement crue dérèglement crue dérèglement crue precedente dérèglement crue dérèglement crue biodiversité disparition espèce biodiversité disparition espèce biodiversité dispariti...,effort particulier industrie carbone acteur fonction émission manière intégralité transition installation éolien mer utilisation énergie empreinte carbone produit empreinte produit tva produit produit baisse tva sanction donnée empreinte carbone commerce carton bouteille air ciel environnement élevage viande eau toilette replanton haie entretenon interdison faison bassin retention eau transiti...
1,1090,vie village moyen chaudière fioul voiture décentralisation relocalisation production véhicule écologie cause transition urgence avis,dérèglement crue pollution pollution air dérèglement crue dérèglement crue réponse production energie dérèglement crue,source problème utilisation centralisation ville désertification augmentation nécessité voiture augmentation ville sanction pays consommation extraction énergie place écologie école place développement niveau utilisation solution énergie fin surconsommation fin bouteille industrie
2,1100,pollution pétrole développement ecologie transport conscience vivre monde contribution pollution préservon maison urgence protection ere priorité pragamatique émotion recyclage agriculture liaison route transport marchandise pauvre,pollution air dérèglement crue consommation agriculture biodiversité disparition espèce biodiversité disparition espèce dérèglement crue biodiversité disparition espèce dérèglement crue dérèglement crue biodiversité disparition espèce problème ensemble puisqu biodiversité disparition espèce pollution acidifcation océan biodiversité problème problème pollution pollution air dérèglement crue,poids lourd exception transport centre ville véhicule véhicule livraison hydrogène expert pétrole an pétrole économie monde chaos choix pétrole pétrole pouvoir hui besoin solution déchet type maximum utilisation travail fusion solution masse hui agriculture subvention arrêt braconnage protection biotope concession zad lande pesticide importation déforestation environnement etat entreprise indi...
3,1110,santé proposition justice ruralité comportement civisme sens pollution air pollution nature santé environnement,dérèglement crue problème pollution air dérèglement crue dérèglement crue biodiversité disparition espèce pollution air dérèglement crue citoyen anture mac pesticide,convaincre sauvetage pose institution occurrence effort tendance besoin argent chose institution fmi argent pays exemple recherche fusion sacré avancée production b timent migre voiture effort conversion offerte france fmi miracle climat entreprise produit fonction coût carbone importation gratuité commun camion france train billet état exemple solution manière agriculture environnement produc...
4,1120,solution pollution travail prise responsabilité développement production consommation réduction dechet amelioration transport gratuité transport hypercentre hydrogene économie pollution entreprise transport transition exemple cite ecologie arme pollution arme pollution agriculture avenir transition solution solution,dérèglement crue pollution air dérèglement crue biodiversité disparition espèce dérèglement crue pollution air dérèglement crue dérèglement crue pollution air biod

In [26]:
grouped_lemm_df.to_csv('grouped_lemmetized.csv')